In [ ]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 447 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import OneHotEncoder

# Loading Data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rajap20/streamlit-example/master/data_final.csv')

In [ ]:
df.head(5)

,Customer_id,Date_disb,pay_type,area_code,pin_code,state,dealer,product_code,tenure,roi,emi,proc_fee,asset_cost,loan_amt,gender,qualification,employ_type,resid_type,age,cibil_score,net_salary,net_irr,fraud
0,0,2/13/2020,ACH,3075,530041,AP,DEALER,SC,0.470588,-0.793282,0.124252,-0.093758,87000,71000,M,OTHERS,SAL,O,0.306122,-0.906781,0.158169,-1.321153,0
1,1,06-02-2020,ADM,3075,532123,AP,DEALER,SC,0.352941,-0.952045,0.083863,-0.572295,61500,51000,F,PG,SAL,O,0.551020,0.163204,1.161336,-1.308269,0
2,2,10-02-2020,ADM,3075,533218,AP,ASC,SC,0.352941,0.318061,0.210079,0.913555,87000,74038,M,SSC,SEP,O,0.530612,NaN,0.910545,0.605033,0
3,3,10-02-2020,ADM,3075,532484,AP,DEALER,SC,0.352941,-1.309262,0.166511,0.649593,84000,69720,M,OTHERS,AGR,O,0.224490,-0.167982,-0.544048,-1.098901,0
4,4,10-02-2020,ADM,3075,530046,AP,DEALER,SC,0.352941,-0.713900,0.092091,-0.572295,61500,52220,F,HSC,SAL,O,0.224490,NaN,-0.343415,-0.986164,0


# Feature Set Selection




In [ ]:
df.columns

Index(['Customer_id', 'Date_disb', 'pay_type', 'area_code', 'pin_code',
       'state', 'dealer', 'product_code', 'tenure', 'roi', 'emi', 'proc_fee',
       'asset_cost', 'loan_amt', 'gender', 'qualification', 'employ_type',
       'resid_type', 'age', 'cibil_score', 'net_salary', 'net_irr', 'fraud'],
      dtype='object')

In [ ]:
x_features = [
              'area_code', 
              'state', 'resid_type', 'net_irr', 'proc_fee',
       'asset_cost', 'loan_amt', 'emi', 'net_salary', 'roi', 'tenure',
       'age']

In [ ]:
cat_features = [
                'area_code', 
                'state', 'resid_type']

In [ ]:
num_features = list(set(x_features) - set(cat_features))
num_features

['asset_cost',
 'emi',
 'loan_amt',
 'tenure',
 'roi',
 'age',
 'proc_fee',
 'net_irr',
 'net_salary']

In [ ]:
df[cat_features] = df[cat_features].astype(object)
df[x_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11702 entries, 0 to 11701
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_code   11702 non-null  object 
 1   state       11702 non-null  object 
 2   resid_type  11698 non-null  object 
 3   net_irr     11702 non-null  float64
 4   proc_fee    11702 non-null  float64
 5   asset_cost  11702 non-null  int64  
 6   loan_amt    11702 non-null  int64  
 7   emi         11702 non-null  float64
 8   net_salary  11702 non-null  float64
 9   roi         11702 non-null  float64
 10  tenure      11702 non-null  float64
 11  age         11698 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 1.1+ MB


In [ ]:
df = df[x_features + ['fraud']].dropna()
df.shape

(11698, 13)

In [ ]:
from category_encoders import OneHotEncoder


ohe_encoder = OneHotEncoder(cols = cat_features,
                            use_cat_names = True)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
ohe_encoder.fit(df)
df_encoded = ohe_encoder.transform(df)

In [ ]:
df_encoded

,area_code_3075.0,area_code_3072.0,area_code_3058.0,area_code_3068.0,area_code_3015.0,area_code_3045.0,area_code_3049.0,area_code_3026.0,area_code_3032.0,area_code_3030.0,area_code_3031.0,area_code_3091.0,area_code_3047.0,area_code_3053.0,area_code_3090.0,area_code_3009.0,area_code_3110.0,area_code_3024.0,area_code_3025.0,area_code_3022.0,area_code_3023.0,area_code_3061.0,area_code_3029.0,area_code_3028.0,area_code_3018.0,area_code_3010.0,area_code_3014.0,area_code_3011.0,area_code_3012.0,area_code_3051.0,area_code_3088.0,area_code_3001.0,area_code_3048.0,area_code_3044.0,area_code_3089.0,area_code_3050.0,area_code_3046.0,area_code_3036.0,area_code_3016.0,area_code_3037.0,...,area_code_3065.0,area_code_3074.0,state_AP,state_AS,state_BR,state_CG,state_DL,state_UP,state_HA,state_GJ,state_HP,state_CH,state_JH,state_JK,state_KA,state_KL,state_MH,state_MP,state_OR,state_TN,state_PY,state_PB,state_RJ,state_WB,state_UC,state_TR,state_MN,resid_type_O,resid_type_R,resid_type_L,net_irr,proc_fee,asset_cost,loan_amt,emi,net_salary,roi,tenure,age,fraud
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1.321153,-0.093758,87000,71000,0.124252,0.158169,-0.793282,0.470588,0.306122,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1.308269,-0.572295,61500,51000,0.083863,1.161336,-0.952045,0.352941,0.551020,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.605033,0.913555,87000,74038,0.210079,0.910545,0.318061,0.352941,0.530612,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1.098901,0.649593,84000,69720,0.166511,-0.544048,-1.309262,0.352941,0.224490,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.986164,-0.572295,61500,52220,0.092091,-0.343415,-0.713900,0.352941,0.224490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11697,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.231391,1.427003,124300,98900,0.258508,-0.243098,0.476825,0.470588,0.122449,0
11698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.279707,1.740351,136643,105900,0.235976,0.910545,0.631619,0.588235,0.285714,0
11699,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.141202,0.586583,61651,52900,0.088631,0.057852,-1.352922,0.352941,0.306122,0
11700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.685559,0.978268,129300,88000,0.219147,-0.092623,0.834042,0.470588,0.306122,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df_encoded,train_size = 0.8,random_state = 100)

## Finding the best model

Using H2O AutoML Library

Details: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/downloading.html?_ga=2.37321956.534948750.1625291360-92496293.1624088300

In [ ]:
# Installing H2O dependencies
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [ ]:
# Installing H2O
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 175.8 MB 24 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.3-py2.py3-none-any.whl size=175832480 sha256=0cad55cb9ca3f0575c89e4755d1719d22db1bcfc6466c44a4818985112af2097
  Stored in directory: /root/.cache/pip/wheels/2a/52/cf/01f6938288f0731963463ff72510db67f9f3076f801038b2f7
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
## prepare data
h2o.init()

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqt8qh9c3
  JVM stdout: /tmp/tmpqt8qh9c3/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpqt8qh9c3/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,2 months
H2O_cluster_name:,H2O_from_python_unknownUser_12ijgy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
encoded_x_features = list(df_encoded.drop('fraud', axis=1).columns)

In [ ]:
train[y]

8796     0
11679    0
8393     1
7411     1
1611     0
        ..
350      1
79       0
8042     0
6939     0
5643     0
Name: fraud, Length: 9358, dtype: int64

In [ ]:
# Identify predictors and response
x = list(train.columns)
y = "fraud"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactors()
test[y] = test[y].asfactors()

In [ ]:
train[y]


8796     0
11679    0
8393     1
7411     1
1611     0
        ..
350      1
79       0
8042     0
6939     0
5643     0
Name: fraud, Length: 9358, dtype: object

In [ ]:
aml = H2OAutoML(max_models=20,
                seed=1,
                stopping_metric='Auto',
                sort_metric='Auto',
                max_runtime_secs = 600,
                exclude_algos = ["GLM", "DeepLearning"])

aml.train(x=x, 
          y='fraud', 
          training_frame=h2o_train) 
#XRT - Extremely Randomized Trees

## Leaderboard